In [1]:
import pandas as pd


df = pd.DataFrame()



In [ ]:
import patoolib


In [5]:
!pwd

/Users/zanchitta/Developer/BugDetectiveAI/Bugdetectiveai/scr/utils


In [4]:
patoolib.extract_archive('../../data/pytracebugs_dataset_v1.rar', outdir='Bugdetectiveai/data/')

INFO patool: Extracting ../../data/pytracebugs_dataset_v1.rar ...


PatoolError: could not find an executable program to extract format rar; candidates are rar,unrar,7z,7zz,7zzs,unar